In [1]:
import os
%pwd

'/home/izam/coding/Sentiment-Analysis/research'

In [2]:
os.chdir('../')
%pwd

'/home/izam/coding/Sentiment-Analysis'

In [3]:
# entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    target_column: str
    vectorizer_path: str

In [4]:
from sentimentAnalysis.constants import *
from sentimentAnalysis.utils.common import read_yaml, create_directories, save_json

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            vectorizer_path = config.vectorizer_path,
           
        )

        return model_evaluation_config

In [6]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sentimentAnalysis.utils.common import load_pickle, save_json, round_batch
from sentimentAnalysis import logger

In [7]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    

    def _eval_metrics(self,actual, pred):
        acc = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        f1 = f1_score(actual, pred)

        return round_batch(acc, precision, recall, f1)

    def evaluate(self):
        df = pd.read_csv(self.config.test_data_path)
        X = df.drop([self.config.target_column], axis=1).iloc[:, 0]
        y = df[self.config.target_column]

        vectorizer = load_pickle(path=Path(self.config.vectorizer_path))
        matrix = vectorizer.transform(X)

        model = load_pickle(path=Path(self.config.model_path))
        pred = model.predict(matrix)

        logger.info(f"predicted {pred.shape[0]} data points")
    
        acc, precision, recall, f1 = self._eval_metrics(y, pred)

        metric = {
            "Accuracy" : acc,
            "Precision" : precision,
            "Recall": recall,
            "F1 score": f1,
        }

        logger.info(f"metrics are - {metric}")

        save_json(path=Path(self.config.metric_file_name), data=metric)

In [8]:
try:
    config = ConfigurationManager()
    get_model_evaluation_config = config.get_model_evaluation_config()
    get_model_evaluation_config = ModelEvaluation(config=get_model_evaluation_config)
    get_model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2023-12-09 21:29:19,980: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-09 21:29:19,981: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-09 21:29:19,983: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-09 21:29:19,984: INFO: common: created directory at: artifacts]
[2023-12-09 21:29:19,984: INFO: common: created directory at: artifacts/model_evaluation]
[2023-12-09 21:29:19,995: INFO: common: pickle file loaded from: artifacts/data_transformation/tfidf_vectorizer.pkl]
[2023-12-09 21:29:20,008: INFO: common: pickle file loaded from: artifacts/model_trainer/model.pkl]
[2023-12-09 21:29:20,009: INFO: 1027808873: predicted 1800 data points]
[2023-12-09 21:29:20,013: INFO: 1027808873: metrics are - {'Accuracy': 0.93, 'Precision': 0.94, 'Recall': 0.91, 'F1 score': 0.93}]
[2023-12-09 21:29:20,014: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
